# Match Winner Prediction - LOL Esports

In [1]:
import os
import pandas as pd

In [2]:
# folder_path = "data/"

# years_data = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".csv")]

# dataframes = [pd.read_csv(year_data) for year_data in years_data]

# # concatenate all df
# original_df = pd.concat(dataframes, ignore_index=True)

In [3]:
# # drop in-game stats and useless Ids
# features_to_keep = ["gameid", "league", "year", "date","split", "playoffs", "patch", "side", "position", "playername", "teamname", "champion", 
#                     "ban1", "ban2", "ban3", "ban4", "ban5", "pick1", "pick2", "pick3", "pick4", "pick5", "result"]

# df = original_df[features_to_keep]

In [4]:
# df.info()

In [5]:
# display(df.head(12))
# print(f"df after droping features : {df.shape}")

## Restructuring the dataFrame.

In [6]:
# # get team rows and player rows.
# team_df = df[df['position'] == 'team'].copy()    # team's info (picks/ban)
# player_df = df[df['position'] != 'team'].copy()    # player's info (name, pisition, ...)

# # pivot
# role_mapping = {
#     'top': 'top_player',
#     'jng': 'jgl_player',
#     'mid': 'mid_player',
#     'bot': 'bot_player',
#     'sup': 'supp_player'
# }

# # for blue:
# blue_players = player_df[player_df['side'] == 'Blue']
# blue_players_pivot = blue_players.pivot(index='gameid', columns='position', values='playername')
# # se the role mapping to pivot.
# blue_players_pivot = blue_players_pivot.rename(columns=role_mapping).reset_index()
# # add prefix blue_
# blue_players_pivot = blue_players_pivot.rename(
#     columns=lambda c: ("blue_" + c) if c != "gameid" else c
# )

# # same for red:
# red_players = player_df[player_df['side'] == 'Red']
# red_players_pivot = red_players.pivot(index='gameid', columns='position', values='playername')
# red_players_pivot = red_players_pivot.rename(columns=role_mapping).reset_index()
# red_players_pivot = red_players_pivot.rename(
#     columns=lambda c: ("red_" + c) if c != "gameid" else c
# )

# # process team rows to extract aggregated team data (picks, bans, ...).
# # For Blue team:
# teams_blue = team_df[team_df['side'] == 'Blue'][[
#     'gameid', 'teamname',
#     'pick1', 'pick2', 'pick3', 'pick4', 'pick5',
#     'ban1', 'ban2', 'ban3', 'ban4', 'ban5'
# ]].rename(columns={
#     'teamname': 'blue_team',
#     'pick1': 'blue_team_pick1',
#     'pick2': 'blue_team_pick2',
#     'pick3': 'blue_team_pick3',
#     'pick4': 'blue_team_pick4',
#     'pick5': 'blue_team_pick5',
#     'ban1': 'blue_team_ban1',
#     'ban2': 'blue_team_ban2',
#     'ban3': 'blue_team_ban3',
#     'ban4': 'blue_team_ban4',
#     'ban5': 'blue_team_ban5'
# })

# # same for red team:
# teams_red = team_df[team_df['side'] == 'Red'][[
#     'gameid', 'teamname',
#     'pick1', 'pick2', 'pick3', 'pick4', 'pick5',
#     'ban1', 'ban2', 'ban3', 'ban4', 'ban5'
# ]].rename(columns={
#     'teamname': 'red_team',
#     'pick1': 'red_team_pick1',
#     'pick2': 'red_team_pick2',
#     'pick3': 'red_team_pick3',
#     'pick4': 'red_team_pick4',
#     'pick5': 'red_team_pick5',
#     'ban1': 'red_team_ban1',
#     'ban2': 'red_team_ban2',
#     'ban3': 'red_team_ban3',
#     'ban4': 'red_team_ban4',
#     'ban5': 'red_team_ban5'
# })

# # extract game data ( including 'date' to sort after).
# game_info_columns = ['gameid', 'league', 'year', 'split', 'patch', 'playoffs', 'date']
# game_info = team_df[game_info_columns].drop_duplicates('gameid')

# # merge all parts together based on 'gameid'.
# merged_df = game_info.merge(teams_blue, on='gameid')\
#                      .merge(teams_red, on='gameid')\
#                      .merge(blue_players_pivot, on='gameid')\
#                      .merge(red_players_pivot, on='gameid')

# # determine the winner. (1 if blue is winner, 0 else)
# blue_result = team_df[team_df['side'] == 'Blue'][['gameid', 'result']].rename(columns={'result': 'is_blue_winner'})

# # merge winner results on games (gameid is deterministic here (can use date ig))
# merged_df = merged_df.merge(blue_result, on='gameid')

# # lastly we sort on 'date' (might be helpful to compute chornological related features like winrate and gamesPlayed)
# merged_df = merged_df.sort_values("date").reset_index(drop=True)


In [7]:
# display(merged_df.head())

how many missing are there for each feature ? (percentage)

In [8]:
# # missing values per feature
# na_percentage = merged_df.isnull().mean()
# display(na_percentage)

#### For the low percetange missing values, we will just drop the rows containing them, but we can't for high percentages.


Let's get the features with very few missing values (less than a threshold we can chose) so we can just drop the corresponding rows. 

In [9]:
# features_less_than_threshold = 0.05 # 5%

# features_less_than_threshold = na_percentage[(na_percentage < 0.05) & (na_percentage != 0)].index.tolist()

# print(features_less_than_threshold)

In [10]:
# # now we drop rows that have missing values in any of these selected features
# merged_df = merged_df.dropna(subset=features_less_than_threshold)

no more features with small missing values.

#### For the splits : we fill with the nearest one. (summer - summer - x - winter : x will be filled with summer).
#### For the picks/bans : fill with imaginary champion named Unknown_champion.

In [11]:
# # for the splits
# high_na_split_feature = ["split"]
# merged_df[high_na_split_feature] = merged_df[high_na_split_feature].fillna(method='ffill').fillna(method='bfill')

In [12]:
# # for the picks/bans
# high_na_percentage_picks_bans_features = ["blue_team_pick1", "blue_team_pick2", "blue_team_pick3", "blue_team_pick4", "blue_team_pick5",
#                                             "red_team_pick1","red_team_pick2", "red_team_pick3", "red_team_pick4", "red_team_pick5",
#                                             "blue_team_ban4", "blue_team_ban5",
#                                             "red_team_ban4", "red_team_ban5"]

# # fill the NAs of these features with Unknown_champion
# merged_df[high_na_percentage_picks_bans_features] = merged_df[high_na_percentage_picks_bans_features].fillna("Unknown_champion")


<!-- one final check, maybe there are missing values but are not signaled with NAN -->

In [13]:
# for col in merged_df.columns:
#     print(merged_df[col].value_counts())
#     print("\n")

<!-- -> unknown team in teams name should be droped too (games of unknown team are useless even more damaging). (will alter the computation of winrates.) -->

In [14]:
# # filter them <-> drop :
# merged_df = merged_df[(merged_df['blue_team'] != 'unknown team') & (merged_df['red_team'] != 'unknown team')]

In [15]:
# # missing values per feature
# na_percentage = merged_df.isnull().mean()
# display(na_percentage)

In [16]:
# merged_df.to_csv("games_aggregated.csv", index=False)
# ;

No more missing values :D

Read the saved csv

In [17]:
processed_df = pd.read_csv("games_aggregated.csv")

In [18]:
processed_df.head()

,gameid,league,year,split,patch,playoffs,date,blue_team,blue_team_pick1,blue_team_pick2,...,blue_jgl_player,blue_mid_player,blue_supp_player,blue_top_player,red_bot_player,red_jgl_player,red_mid_player,red_supp_player,red_top_player,is_blue_winner
0,TRLH3/33,EU LCS,2014,Spring,3.15,0,2014-01-14 17:52:02,Fnatic,Unknown_champion,Unknown_champion,...,Cyanide,xPeke,YellOwStaR,sOAZ,Genja,Diamondprox,Alex Ich,Edward,Darien,1
1,TRLH3/44,EU LCS,2014,Spring,3.15,0,2014-01-14 19:16:29,Copenhagen Wolves,Unknown_champion,Unknown_champion,...,Amazing,cowTard,Unlimited,YoungBuck,Tabzz,Shook,Froggen,Nyph,Wickd,1
2,TRLH3/76,EU LCS,2014,Spring,3.15,0,2014-01-14 21:28:45,Alliance,Unknown_champion,Unknown_champion,...,Shook,Froggen,Nyph,Wickd,Rekkles,Cyanide,xPeke,YellOwStaR,sOAZ,0
3,TRLH3/85,EU LCS,2014,Spring,3.15,0,2014-01-14 22:37:18,Gambit Gaming,Unknown_champion,Unknown_champion,...,Diamondprox,Alex Ich,Edward,Darien,FORG1VEN,Amazing,cowTard,Unlimited,YoungBuck,1
4,TRLH3/10072,EU LCS,2014,Spring,3.15,0,2014-01-14 23:48:12,SK Gaming,Unknown_champion,Unknown_champion,...,Svenskeren,Jesiz,nRated,fredy122,Creaton,Araneae,Kerp,Jree,Kev1n,0


In [19]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80004 entries, 0 to 80003
Data columns (total 40 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gameid            80004 non-null  object 
 1   league            80004 non-null  object 
 2   year              80004 non-null  int64  
 3   split             80004 non-null  object 
 4   patch             80004 non-null  float64
 5   playoffs          80004 non-null  int64  
 6   date              80004 non-null  object 
 7   blue_team         80004 non-null  object 
 8   blue_team_pick1   80004 non-null  object 
 9   blue_team_pick2   80004 non-null  object 
 10  blue_team_pick3   80004 non-null  object 
 11  blue_team_pick4   80004 non-null  object 
 12  blue_team_pick5   80004 non-null  object 
 13  blue_team_ban1    80004 non-null  object 
 14  blue_team_ban2    80004 non-null  object 
 15  blue_team_ban3    80004 non-null  object 
 16  blue_team_ban4    80004 non-null  object

## Feature engineering

We will calculate the winrate for each team:
- helps alot with target encoding (e.g. for teams) that highly contributes to the final prediction.

we will add two features, blue_team_winrate and red_team_winrate

In [20]:
# setting the list of featres

features_to_drop = ["gameid", "date"] # no info (high cardinality)

numerical_features = ["year", "playoffs", "is_blue_winner"] # patch is float but will be treated as object

In [ ]:
# winrate
# haaving the dataframe sorted chronologically helps here.

# saves
blue_winrates = []
red_winrates = []
blue_games_played = []
red_games_played = []
blue_H2H_red = []
red_H2H_blue = []

# dictionaries to maintain cumulative records
team_records = {}  # { team name : {'wins': int, 'games': int} }
h2h_records = {}   # { (blue_team, red_team) : {'wins': int, 'games': int} }

# loop over each game (<-> each row).
for index, row in processed_df.iterrows():
    blue_team = row['blue_team']
    red_team = row['red_team']
    
    # --- Cumulative team winrates/gamesPlayed ---
    blue_record = team_records.get(blue_team, {'wins': 0, 'games': 0})
    red_record = team_records.get(red_team, {'wins': 0, 'games': 0})
    
    blue_wr = blue_record['wins'] / blue_record['games'] if blue_record['games'] > 0 else 0.5
    red_wr = red_record['wins'] / red_record['games'] if red_record['games'] > 0 else 0.5
    
    blue_gp = blue_record['games']
    red_gp = red_record['games']
    
    blue_winrates.append(blue_wr)
    red_winrates.append(red_wr)
    blue_games_played.append(blue_gp)
    red_games_played.append(red_gp)
    
    # --- blue_H2H_red and red_H2H_blue---
    key = (blue_team, red_team)
    h2h_rec = h2h_records.get(key, {'wins': 0, 'games': 0})
    
    blue_h2h_count = h2h_rec['wins'] if h2h_rec['games'] > 0 else 0
    blue_H2H_red.append(blue_h2h_count)
    
    # red_H2H_blue, red's wins are games played minus blue's wins.
    red_h2h_count = (h2h_rec['games'] - h2h_rec['wins']) if h2h_rec['games'] > 0 else 0
    red_H2H_blue.append(red_h2h_count)
    
    # --- Update teams records ---
    team_records.setdefault(blue_team, {'wins': 0, 'games': 0})
    team_records.setdefault(red_team, {'wins': 0, 'games': 0})
    team_records[blue_team]['games'] += 1
    team_records[red_team]['games'] += 1
    if row['is_blue_winner'] == 1:
        team_records[blue_team]['wins'] += 1
    else:
        team_records[red_team]['wins'] += 1
    
    # --- Update H2H records ---
    h2h_records.setdefault(key, {'wins': 0, 'games': 0})
    h2h_records[key]['games'] += 1
    if row['is_blue_winner'] == 1:
        h2h_records[key]['wins'] += 1

# add the computed features as new features.
processed_df['blue_team_winrate'] = blue_winrates
processed_df['red_team_winrate'] = red_winrates
processed_df['blue_team_games_played'] = blue_games_played
processed_df['red_team_games_played'] = red_games_played
processed_df['blue_H2H_red'] = blue_H2H_red
processed_df['red_H2H_blue'] = red_H2H_blue

In [25]:
processed_df.head()

,gameid,league,year,split,patch,playoffs,date,blue_team,blue_team_pick1,blue_team_pick2,...,red_mid_player,red_supp_player,red_top_player,is_blue_winner,blue_team_winrate,red_team_winrate,blue_team_games_played,red_team_games_played,blue_H2H_red,red_H2H_blue
0,TRLH3/33,EU LCS,2014,Spring,3.15,0,2014-01-14 17:52:02,Fnatic,Unknown_champion,Unknown_champion,...,Alex Ich,Edward,Darien,1,0.5,0.5,0,0,0,0
1,TRLH3/44,EU LCS,2014,Spring,3.15,0,2014-01-14 19:16:29,Copenhagen Wolves,Unknown_champion,Unknown_champion,...,Froggen,Nyph,Wickd,1,0.5,0.5,0,0,0,0
2,TRLH3/76,EU LCS,2014,Spring,3.15,0,2014-01-14 21:28:45,Alliance,Unknown_champion,Unknown_champion,...,xPeke,YellOwStaR,sOAZ,0,0.0,1.0,1,1,0,0
3,TRLH3/85,EU LCS,2014,Spring,3.15,0,2014-01-14 22:37:18,Gambit Gaming,Unknown_champion,Unknown_champion,...,cowTard,Unlimited,YoungBuck,1,0.0,1.0,1,1,0,0
4,TRLH3/10072,EU LCS,2014,Spring,3.15,0,2014-01-14 23:48:12,SK Gaming,Unknown_champion,Unknown_champion,...,Kerp,Jree,Kev1n,0,0.5,0.5,0,0,0,0


## Pre-Process

Let's check the unique values for each feature

In [23]:
processed_df.nunique()

gameid                    80004
league                      116
year                         12
split                        53
patch                       244
playoffs                      2
date                      79850
blue_team                  1851
blue_team_pick1             162
blue_team_pick2             169
blue_team_pick3             169
blue_team_pick4             171
blue_team_pick5             170
blue_team_ban1              170
blue_team_ban2              169
blue_team_ban3              169
blue_team_ban4              169
blue_team_ban5              170
red_team                   1857
red_team_pick1              165
red_team_pick2              167
red_team_pick3              170
red_team_pick4              170
red_team_pick5              171
red_team_ban1               170
red_team_ban2               170
red_team_ban3               170
red_team_ban4               171
red_team_ban5               171
blue_bot_player            2273
blue_jgl_player            2336
blue_mid